<a href="https://colab.research.google.com/github/Jisang-hwang93/BOK_Report_Team1/blob/master/%EC%9D%98%EC%82%AC%EB%A1%9D_%EC%84%B9%EC%85%98%EB%B6%84%EB%A6%AC_%EC%8B%9C%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import platform
import pandas as pd
import re
from datetime import datetime, timedelta
import csv

In [ ]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [ ]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [ ]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(tokens)
                
    return tokens+ ngrams

In [ ]:
bok = pd.read_csv('bok_text_date.csv', index_col=[0], header=0, sep=',')
bok.drop([300,301,302,303,304,305,306,307,308,309], inplace=True)  #300번 이후 텍스트 인코딩문제(?)가 있어 일단 제거함
boks = bok['text'][::-1]
boks_date = bok['date'][::-1]

In [ ]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join

    
def preprocessing(source_folder="./", output_file="bok_minutes_again.csv") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
#     txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
#     txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""

    df["ngram"] = ""
    

    for idx, bok in enumerate(boks) :
        sections, section_texts = preprocess_minutes(bok)

        print(boks_date[idx])

        try:
            ngrams = []
            for section_text in section_texts[1:3] :
                for sentence in section_text :
                    ngrams.append(','.join(text2ngram(sentence)))

            #print('@@@'.join(ngrams))
            #break

            df.loc[len(df)] = [boks_date[idx],
                               bok,
                               '@@@'.join(section_texts[0]),
                               '@@@'.join(section_texts[1]),
                               '@@@'.join(section_texts[2]),
                               '@@@'.join(section_texts[3]),
                               '@@@'.join(section_texts[4]),
                               '@@@'.join(section_texts[5]),
                               len(section_texts[0]),
                               len(section_texts[1]),
                               len(section_texts[2]),
                               len(section_texts[3]),
                               len(section_texts[4]),
                               len(section_texts[5]),
                               '@@@'.join(ngrams)]
        except:
            print(boks_date[idx], "error")

        df.to_csv("bok_minutes_again.csv", index=False)

        print(df.shape)
preprocessing()

2020.07.10
(1, 15)
2020.06.30
(2, 15)
2020.06.16
(3, 15)
2020.06.02
(4, 15)
2020.05.06
(5, 15)
2020.04.28
(6, 15)
2020.04.14
(7, 15)
2020.03.31
(8, 15)
2020.03.31
(9, 15)
2020.03.17
(10, 15)
2020.02.04
(11, 15)
2020.01.10
(12, 15)
2019.12.31
(13, 15)
2019.12.17
(14, 15)
2019.11.05
(15, 15)
2019.10.11
(16, 15)
2019.09.17
(17, 15)
2019.08.27
(18, 15)
2019.08.06
(19, 15)
2019.07.09
(20, 15)
2019.06.18
(21, 15)
2019.05.28
(22, 15)
2019.05.07
(23, 15)
2019.04.12
(24, 15)
2019.03.19
(25, 15)
2019.03.05
(26, 15)
2019.02.12
(27, 15)
2019.02.07
(28, 15)
2019.01.15
(29, 15)
2019.01.08
(30, 15)
2018.12.26
(31, 15)
2018.12.18
(32, 15)
2018.11.27
(33, 15)
2018.11.06
(34, 15)
2018.10.10
(35, 15)
2018.09.18
(36, 15)
2018.08.14
(37, 15)
2018.07.31
(38, 15)
2018.07.06
(39, 15)
2018.06.12
(40, 15)
2018.05.15
(41, 15)
2018.05.02
(42, 15)
2018.04.17
(43, 15)
2018.03.27
(44, 15)
2018.03.20
(45, 15)
2018.02.27
(46, 15)
2018.02.06
(47, 15)
2018.01.16
(48, 15)
2018.01.02
(49, 15)
2017.12.26
(50, 15)
2017.12.1

In [4]:
tmp = pd.read_csv("/content/drive/My Drive/for_weekend/bok_minutes_again.csv", index_col=[0], header=0, sep=',')
tmp 


,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,ngram
date,,,,,,,,,,,,,,
2020.07.10,금융통화위원회 의사록\n\n2006년도 제2차 회의\n\n1. 일 \n\n ...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2020.06.30,금융통화위원회 의사록\n\n2006년도 제3차 회의\n\n1. 일 \n\n ...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2020.06.16,금융통화위원회 의사록\n\n2006년도 제4차 회의\n\n1. 일 \n\n ...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2020.06.02,금융통화위원회 의사록\n\n2006년도 제6차 회의\n\n1. 일 \n\n ...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2020.05.06,금융통화위원회 의사록\n\n 김 태 동 \n\n 위 원 \n\n \n\n...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006.05.09,2020년도 제9차\n\n금융통화위원회(임시) 의사록\n\n한 국 은 행\n\n 1...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2006.04.25,2020년도 제11차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n ...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2006.03.28,2020년도 제12차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n ...,일부 위원은 코로나19 사태의 전개양상을 둘러싼 불확실성이 매우 높은 상황 에서 ...,일부 위원은 코로나19 사태 이전 풍부한 유동성에 힘입어 미 증시가 사상 최 고치...,NaN,일부 위원은 기준금리의 실효하한에 대해 다양한 견해가 있고 그 수준도 특 정하기가...,일부 위원은 이번 회의에서 기준금리를 0.50%로 25bp 인하하여 우리 경제에 ...,NaN,63,74,0,28,98,0,"코로나/NNG,사태/NNG,이전/NNG,풍부/NNG,유동성/NNG,힘입/VV,미/N..."


In [5]:
pd.DataFrame(tmp['ngram'].dropna())

,ngram
date,
2018.12.26,"부채/NNG,대외/NNG,부채/NNG,증가/NNG,국제/NNG,유가/NNG,상승/N..."
2018.02.27,"채권보증/NNG,기관/NNG,신용등급/NNG,하락/NNG,국제/NNG,금융시장/NN..."
2018.02.06,"물량/NNG,집중/NNG,롤오버/NNG,알려/VV,점검/NNG,필요/NNG,있/VA..."
2018.01.02,"fed/NNG,금리/NNG,인하/NNG,유동성/NNG,공급/NNG,확대/NNG,조치..."
2017.12.26,"국제/NNG,금융시장/NNG,신용위험/NNG,완화/NNG,되/XSV,재정거래/NNG..."
...,...
2006.10.10,"fed/NNG,발표/NNG,하/VV,금융안정보고서/NNG,따르/VV,주식선물/NNG..."
2006.08.22,"글로벌/NNG,펀드/NNG,자금/NNG,유출입/NNG,동향/NNG,살펴보/VV,지역..."
2006.08.08,"외국인/NNG,주식/NNG,투자자금/NNG,움직임/VV,살펴보/VV,패시브/NNG,..."


In [ ]:
#133개 빼고는 다 오류남 .. 문제 해결 필요

In [ ]:
import pandas as pd
df = pd.read_csv("./data/minutes/minutes_preprocessed.csv")
#df['ngram'][0]

In [ ]:
df

In [ ]:
df[['date','Economic Situation count', 'Foreign Currency count','Financial Markets count','Monetary Policy count','Participant Views count','Government View count']].to_csv("./results/minutes_daily_count.csv", index = False)